In [7]:
pip install llmcompressor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 253.4/253.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 169.0/169.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 105.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 105.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 24.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 585.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
pip install -U datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 22.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 18.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pl

In [2]:
pip list | grep compress

compressed-tensors                    0.10.2
llmcompressor                         0.6.0


In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM

MODEL_ID = "deepseek-ai/DeepSeek-R1-0528-Qwen3-8B"
model = AutoModelForCausalLM.from_pretrained(MODEL_ID, torch_dtype="auto")
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [5]:
from datasets import load_dataset

NUM_CALIBRATION_SAMPLES=1024
MAX_SEQUENCE_LENGTH=2048

# Load dataset
ds = load_dataset("HuggingFaceH4/ultrachat_200k", split=f"train_sft[:{NUM_CALIBRATION_SAMPLES}]")
ds = ds.shuffle(seed=42)

# Preprocess the data into the format the model is trained with
def preprocess(example):
    return {"text": tokenizer.apply_chat_template(example["messages"], tokenize=False,)}
ds = ds.map(preprocess)

print(f"Columns in the dataset after pro-processing: {ds.column_names}")

top_1_row = ds.select(range(1))
for row in top_1_row:
    print(f"Top row data for the dataset after pro-processing: {row}")

# Tokenize the data (be careful with bos tokens - we need add_special_tokens=False since the chat_template already added it)
def tokenize(sample):
    return tokenizer(sample["text"], padding=False, max_length=MAX_SEQUENCE_LENGTH, truncation=True, add_special_tokens=False)
ds = ds.map(tokenize, remove_columns=ds.column_names)

print(f"Columns in the dataset after tokenizing and removing default dataset columns: {ds.column_names}")

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Columns in the dataset after pro-processing: ['prompt', 'prompt_id', 'messages', 'text']
Top row data for the dataset after pro-processing: {'prompt': "Write a 500-word fictional story in third person omniscient point of view about a middle-aged man named Tom who sets out for a solo hike in the Appalachian Mountains, but gets lost due to a sudden storm. Incorporate themes of survival, resourcefulness, and inner strength in the story. Use vivid descriptions and sensory details to create an atmospheric setting and build tension. End the story with a realistic resolution that reflects the character's growth and transformation.", 'prompt_id': 'b5d7b223287fa2d68f67a34ef0ff1ba6f137a765cd19bbc1f9fe8ff16e28615e', 'messages': [{'content': "Write a 500-word fictional story in third person omniscient point of view about a middle-aged man named Tom who sets out for a solo hike in the Appalachian Mountains, but gets lost due to a sudden storm. Incorporate themes of survival, resourcefulness, and in

Map:   0%|          | 0/1024 [00:00<?, ? examples/s]

Columns in the dataset after tokenizing and removing default dataset columns: ['input_ids', 'attention_mask']


In [ ]:
from llmcompressor import oneshot
from llmcompressor.modifiers.quantization import GPTQModifier

# Configure the quantization algorithm to run
recipe = GPTQModifier(targets="Linear", scheme="W4A16", ignore=["lm_head"])

# Apply quantization
oneshot(
    model=model, dataset=ds,
    recipe=recipe,
    max_seq_length=MAX_SEQUENCE_LENGTH,
    num_calibration_samples=NUM_CALIBRATION_SAMPLES,
)

# Save to disk compressed
SAVE_DIR = MODEL_ID.rstrip("/").split("/")[-1] + "-W4A16-G128"
model.save_pretrained(SAVE_DIR, save_compressed=True)
tokenizer.save_pretrained(SAVE_DIR)

Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


2025-07-04T06:17:03.834053+0000 | reset | INFO - Compression lifecycle reset
2025-07-04T06:17:03.838233+0000 | from_modifiers | INFO - Creating recipe from modifiers
2025-07-04T06:17:03.955922+0000 | initialize | INFO - Compression lifecycle initialized for 1 modifiers
2025-07-04T06:17:03.956883+0000 | IndependentPipeline | INFO - Inferred `SequentialPipeline` for `GPTQModifier`


(1/37): Calibrating: 100%|██████████| 1024/1024 [06:20<00:00,  2.69it/s]

2025-07-04T06:23:33.812655+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.q_proj using 1024 samples


2025-07-04T06:23:37.597812+0000 | compress | METRIC - time 3.78s
2025-07-04T06:23:37.601369+0000 | compress | METRIC - error 7.21
2025-07-04T06:23:37.604162+0000 | compress | METRIC - GPU 0 | usage: 30.01% | total memory: 16 GB
2025-07-04T06:23:37.605035+0000 | compress | METRIC - Compressed module size: 33.947648 MB
2025-07-04T06:23:37.607943+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.k_proj using 1024 samples
2025-07-04T06:23:41.009961+0000 | compress | METRIC - time 3.40s
2025-07-04T06:23:41.011607+0000 | compress | METRIC - error 2.36
2025-07-04T06:23:41.012741+0000 | compress | METRIC - GPU 0 | usage: 30.01% | total memory: 16 GB
2025-07-04T06:23:41.013612+0000 | compress | METRIC - Compressed module size: 8.486912 MB
2025-07-04T06:23:41.014749+0000 | compress_modules | INFO - Quantizing model.layers.0.self_attn.v_proj using 1024 samples
2025-07-04T06:23:43.785308+0000 | compress | METRIC - time 2.77s
2025-07-04T06:23:43.786885+0000 | compress | METRIC - 

(1/37): Propagating: 100%|██████████| 1024/1024 [04:04<00:00,  4.19it/s]


In [ ]:
pip install huggingface_hub

In [ ]:
from huggingface_hub import login

login(token="<<your-token>>") # Replace with your actual token

In [ ]:
from huggingface_hub import HfApi

api = HfApi()
repo_id = "balakriz/DeepSeek-R1-0528-Qwen3-8B-W4A16-G128" # Replace with your desired repo ID
api.create_repo(repo_id, exist_ok=True) # Create the repository if it doesn't exist

local_model_path = "./DeepSeek-R1-0528-Qwen3-8B-W4A16-G128"

# Push the model and tokenizer to the repository
api.upload_folder(
    folder_path=local_model_path,
    repo_id=repo_id,
    repo_type="model",
)